# Comparing annotations of *Crepis callicephala* and *C. purpurea*
The manually corrected genbank files are used. The annotations should be corrected before submitting to databases.

In [ ]:
import pandas as pd
import numpy as np
from Bio import SeqRecord, SeqIO, SeqFeature

# variables
c_cal = "plastomes/in/C_callicephala_curated_1_cleaned_meta.gb"
c_pur = "plastomes/in/Crepis_purpurea_edited+.gb"

regions = {}
dataset = []

# Do initial parsing
for acc in c_cal, c_pur:
    with open(acc, "r") as handle:
        for record in SeqIO.parse(handle, "genbank"):
            features = record.features
            # check if there features other than 'source'
            if len(features) > 1:
                for feature in features:
                    # check for genome regions
                    if feature.type in ['misc_feature', 'repeat_region']:
                        label = feature.qualifiers.get('note', "")[0].split()[-1].strip("()")
                        regions[label] = [int(feature.location.start), int(feature.location.end)]
                # filling the dataframe
                organism = record.annotations.get('organism', "")
                for feature in features:
                    if feature.type == 'gene':
                        gene = feature.qualifiers.get('gene', "")
                        start = int(feature.location.start)
                        end = int(feature.location.end)

                        for region in regions.items():
                            ranges = region[-1]
                            if ranges[0] <= start < ranges[-1]:
                                #print(f'{region[0]}\t{gene}')
                                spanned = False
                                if end >= ranges[-1]:
                                    spanned = True
                                dataset.append((organism, gene[0], region[0], start, end, len(feature), spanned))

df = pd.DataFrame(dataset, columns=['organism', 'gene', 'region', 'start', 'end', 'length', 'spanned'])
display(df)

,organism,gene,region,start,end,length,spanned
0,Crepis callicephala,trnH-GUG,LSC,2,77,75,False
1,Crepis callicephala,psbA,LSC,474,1536,1062,False
2,Crepis callicephala,trnK,LSC,1758,4358,2600,False
3,Crepis callicephala,matK,LSC,2079,3591,1512,False
4,Crepis callicephala,trnK-UUU,LSC,4320,4357,37,False
...,...,...,...,...,...,...,...
266,Crepis purpurea,ycf2,IRA,140899,147754,6855,False
267,Crepis purpurea,trnM-CAU | trnM,IRA,147865,147940,75,False
268,Crepis purpurea,rpl23,IRA,148104,148386,282,False
269,Crepis purpurea,rpl2,IRA,148404,149894,1490,False


Comparing repeated features.

In [35]:
df_repeats = df[df['region'].isin(['IRA', 'IRB'])]

pivot_table = df_repeats.pivot_table(
    values='length',
    index='gene',
    columns=['organism', 'region'],
    aggfunc='first'  # or 'mean', 'sum', etc. depending on your needs
)

# Reorder levels in the columns
#pivot_table = df2.reorder_levels([1, 0], axis=1)

# Sort the columns
pivot_table = pivot_table.sort_index(axis=1)

# Display the pivot table
display(pivot_table)


organism                          Crepis callicephala         Crepis purpurea  \
region                                            IRA     IRB             IRA   
gene                                                                            
ndhB                                           2202.0  2202.0          2202.0   
ndhF                                              NaN  2226.0             NaN   
rpl2                                           1490.0  1490.0          1490.0   
rpl23                                           282.0   282.0           282.0   
rps19-fragment                                   60.0     NaN            55.0   
rps7                                            468.0   468.0           468.0   
rrn16                                          1490.0  1490.0          1490.0   
rrn23                                          2810.0  2810.0          2810.0   
rrn4.5                                          103.0   103.0           103.0   
rrn5                                              NaN   116.0           121.0   
rrn5S                                           122.0     NaN             NaN   
trnA-UGC                                        893.0   893.0             NaN   
trnA-UGC | trnA                                   NaN     NaN           893.0   
trnI-GAU                                        834.0   834.0             NaN   
trnI-GAU | trnE-UUC | trnE | trnI                 NaN     NaN           836.0   
trnL-CAA                                         81.0    81.0             NaN   
trnL-CAA | trnL-fragment                          NaN     NaN            83.0   
trnM                                             75.0    74.0             NaN   
trnM-CAU                                          NaN     NaN             NaN   
trnM-CAU | trnM                                   NaN     NaN            75.0   
trnN-GUU                                         72.0    72.0            74.0   
trnR-ACG                                         74.0    74.0            75.0   
trnR-ACG | trnR                                   NaN     NaN             NaN   
trnV-GAC                                         73.0    72.0            72.0   
trnV-GAC | trnV                                   NaN     NaN             NaN   
ycf1                                              NaN   480.0             NaN   
ycf1 fragment                                     NaN     NaN             NaN   
ycf15                                           192.0   192.0           192.0   
ycf2                                           6855.0  6855.0          6855.0   

organism                                   
region                                IRB  
gene                                       
ndhB                               2202.0  
ndhF                               2226.0  
rpl2                               1490.0  
rpl23                               282.0  
rps19-fragment                        NaN  
rps7                                468.0  
rrn16                              1490.0  
rrn23                              2810.0  
rrn4.5                              103.0  
rrn5                                  NaN  
rrn5S                               122.0  
trnA-UGC                              NaN  
trnA-UGC | trnA                     893.0  
trnI-GAU                              NaN  
trnI-GAU | trnE-UUC | trnE | trnI   836.0  
trnL-CAA                              NaN  
trnL-CAA | trnL-fragment             83.0  
trnM                                  NaN  
trnM-CAU                             75.0  
trnM-CAU | trnM                       NaN  
trnN-GUU                             74.0  
trnR-ACG                              NaN  
trnR-ACG | trnR                      75.0  
trnV-GAC                              NaN  
trnV-GAC | trnV                      72.0  
ycf1                                  NaN  
ycf1 fragment                       480.0  
ycf15                               192.0  
ycf2                               6855.0

Compare features within LSC

In [55]:
df_target = df[df['region'].isin(['LSC'])]

non_tRNA = df_target[~df_target['gene'].str.startswith('trn')]
tRNA = df_target[df_target['gene'].str.startswith('trn')]

pt_tRNA = tRNA.pivot_table(
    values='length',
    index='gene',
    columns=['organism'],
    aggfunc='first'  # or 'mean', 'sum', etc. depending on your needs
)

# Sort the columns
pt_tRNA = pt_tRNA.sort_index(axis=1)

pt_nontRNA = non_tRNA.pivot_table(
    values='length',
    index='gene',
    columns=['organism'],
    aggfunc='first'  # or 'mean', 'sum', etc. depending on your needs
)

# Sort the columns
pt_nontRNA = pt_nontRNA.sort_index(axis=1)


# Display the pivot table
#display(pt_tRNA, pt_nontRNA)
#with pd.option_context('display.max_rows', None,):
#    display(pt_nontRNA)

# Find differences
diff_nontRNA = pt_nontRNA['Crepis callicephala'] != pt_nontRNA['Crepis purpurea']
print("Genes with different values:")
display(pt_nontRNA[diff_nontRNA])

diff_tRNA = pt_tRNA['Crepis callicephala'] != pt_tRNA['Crepis purpurea']
print("tRNA genes with different values:")
display(pt_tRNA[diff_tRNA])

Genes with different values:


organism,Crepis callicephala,Crepis purpurea
gene,,
petD,525,1183
psaA,2253,1776
rpl16,1445,1440
rpl22,465,540
rpoC1,2799,2520
rpoC2,4125,2274
rps16,1120,1125


tRNA genes with different values:


organism,Crepis callicephala,Crepis purpurea
gene,,
trnC-GCA,72.0,NaN
trnC-GCA | trnC,NaN,72.0
trnD-GUC,74.0,77.0
trnE-UUC,NaN,74.0
trnE-UUC | trnE,72.0,NaN
trnF-GAA,70.0,NaN
trnF-GAA | trnF,NaN,74.0
trnG-GCC | trnG,NaN,72.0
trnG-GCC | trnG-UCC,71.0,NaN


SSC features with different values

In [56]:
df_target = df[df['region'].isin(['SSC'])]

non_tRNA = df_target[~df_target['gene'].str.startswith('trn')]
tRNA = df_target[df_target['gene'].str.startswith('trn')]

pt_tRNA = tRNA.pivot_table(
    values='length',
    index='gene',
    columns=['organism'],
    aggfunc='first'  # or 'mean', 'sum', etc. depending on your needs
)

# Sort the columns
pt_tRNA = pt_tRNA.sort_index(axis=1)

pt_nontRNA = non_tRNA.pivot_table(
    values='length',
    index='gene',
    columns=['organism'],
    aggfunc='first'  # or 'mean', 'sum', etc. depending on your needs
)

# Sort the columns
pt_nontRNA = pt_nontRNA.sort_index(axis=1)


# Display the pivot table
#display(pt_tRNA, pt_nontRNA)
#with pd.option_context('display.max_rows', None,):
#    display(pt_nontRNA)

# Find differences
diff_nontRNA = pt_nontRNA['Crepis callicephala'] != pt_nontRNA['Crepis purpurea']
print("Genes with different values:")
display(pt_nontRNA[diff_nontRNA])

diff_tRNA = pt_tRNA['Crepis callicephala'] != pt_tRNA['Crepis purpurea']
print("tRNA genes with different values:")
display(pt_tRNA[diff_tRNA])

Genes with different values:


organism,Crepis callicephala,Crepis purpurea
gene,,
ndhA,2147,2146
ndhD,1503,1521
ycf1,4970,4973


tRNA genes with different values:


organism,Crepis callicephala,Crepis purpurea
gene,,
trnL-UAG,80.0,NaN
trnL-UAG | trnL,NaN,80.0


In [1]:
def clean_trna_labels(in_label: str):
    label = None
    #print("initial:", in_label)
    if "trn" in str(in_label).lower():
        if not " | " in str(in_label):
            label = in_label
        else:
            labels = in_label.split(" | ")
            triplet_flag = False
            fragment_flag = False
            marked_by_triplet = []
            for i, x in enumerate(labels):
                #print(i, "x:", x)
                if len(x.split("-")[-1]) == 3:
                    triplet_flag = True
                    marked_by_triplet.append(x)
                if "fragment" in x:
                    fragment_flag = True
            if len(marked_by_triplet) == 1:
                label = labels[0]
            else:
                concise = []
                for idx, y in enumerate(marked_by_triplet):
                    #print(idx, "y", y)
                    if len(y.split("-")[0]) > 4:
                        pass
                    else:
                        concise.append(y)
                if len(concise) == 1:
                    label = concise[0]
                else:
                    #print(f'Manual checking needed: {concise}')
                    # fixing case of trnS-SER
                    for z, a in enumerate(concise):
                        #print(z, "a:", a)
                        suf = a.split("-")[-1]
                        for b in suf:
                            if not b.upper() in "ATGCU":
                                #print("omitted:", b, a)
                                pass
                            else:
                                # ensure assigning the first instanse 
                                # from the concise list to label var
                                if not label == None:
                                    pass
                                else:
                                    label = a
                                    break

            if fragment_flag == True:
                if not "fragment" in label:
                    label = f'{label}-fragment'

        #print(label)
        return label
    
    else:
        return in_label

In [35]:
print(clean_trna_labels("trnI-GAU | trnE-UUC | trnE | trnI	"))

trnI-GAU


In [2]:
def normalize_rrna_label(in_label: str):
    letters = list(in_label)
    terminal = letters[-1]
    if terminal.upper() == "S" and "rrn" in in_label:
        label = in_label[:-1]
    else:
        label = in_label
    
    return label

print(normalize_rrna_label("rra4.5S"))

rra4.5S


In [3]:
def normalize_fragment_label(in_label: str):
    label = None
    if "fragment" in in_label:
        if "-fragment" in in_label:
            shorten = in_label.split("-fragment")[0]
            label = f'{shorten} fragment'
        elif not " " in in_label:
            shorten = in_label.split("fragment")[0]
            label = f'{shorten} fragment'
        else:
            label = in_label
    else:
        label = in_label

    return label

print(normalize_fragment_label("prs19 fragment"))

prs19 fragment


In [4]:
def normalize_fragment_ycf(in_label: str, in_region: str):
    if not "5" in in_label and "ycf" in in_label:
        if in_region == "SSC" or in_region == "LSC":
            label = in_label
        else:
            if "fragment" in in_label:
                label = normalize_fragment_label(in_label)
            elif "ycf1" in in_label:
                label = f'{in_label} fragment'
            else:
                label = in_label
    else:
        label = in_label

    return label

test = "ycf1"
region = "SSC"
print(normalize_fragment_ycf(test, region))

ycf1


### Normalize labels.
#### Iteration 1

In [5]:
def normalize_labels(in_label: str | None = None, region: str | None = None, exception: str | list | None = None) -> str:
    label = None
    #if "trn" in in_label:
    #    label = clean_trna_labels(in_label)
    #elif "rrn" in in_label:
    #    label = normalize_rrna_label(in_label)
    #elif "fragment" in in_label:
    #    label = normalize_fragment_label(in_label)
    #elif "ycf" in in_label:
    #    label = normalize_fragment_ycf(in_label, region)
    label = clean_trna_labels(in_label)
    label = normalize_rrna_label(label)
    label = normalize_fragment_label(label)
    label = normalize_fragment_ycf(label, region)

    if not exception:
        pass
    else:
        if type(exception) == list:
            for exp in exception:
                if exp == in_label or exp == label:
                    label = label.split(" fragment")[0]
                else:
                    pass
        elif type(exception) == str:
            exp = exception
            if exp == in_label or exp == label:
                label = label.split(" fragment")[0]
            else:
                pass
        else:
             pass


    if not label:
        return in_label
    else:
        return label
    
#test = "trnS-CGA | trnG | trnS-SER | trnS"
test = "trnT-GGU | trnT-fragment"
region = "IRB"
exception = ["trnL-CAA fragment", "trnT-GGU | trnT-fragment"]
print(normalize_labels(test, region, exception))

trnT-GGU


In [6]:
import pandas as pd
import numpy as np
from Bio import SeqRecord, SeqIO, SeqFeature

# variables
c_cal = "plastomes/in/C_callicephala_curated_1_cleaned_meta.gb"
c_pur = "plastomes/in/Crepis_purpurea_edited+.gb"

exception = [
    "trnL-CAA fragment",
    "trnT-GGU | trnT-fragment",
]

regions = {}
dataset = []

# Do initial parsing
for acc in c_cal, c_pur:
    with open(acc, "r") as handle:
        for record in SeqIO.parse(handle, "genbank"):
            features = record.features
            # check if there features other than 'source'
            if len(features) > 1:
                for feature in features:
                    # check for genome regions
                    if feature.type in ['misc_feature', 'repeat_region']:
                        label = feature.qualifiers.get('note', "")[0].split()[-1].strip("()")
                        regions[label] = [int(feature.location.start), int(feature.location.end)]
                # filling the dataframe
                organism = record.annotations.get('organism', "")
                for feature in features:
                    if feature.type == 'gene':
                        gene = feature.qualifiers.get('gene', "")
                        start = int(feature.location.start)
                        end = int(feature.location.end)

                        for region in regions.items():
                            ranges = region[-1]
                            if ranges[0] <= start < ranges[-1]:
                                #print(f'{region[0]}\t{gene}')
                                spanned = False
                                if end >= ranges[-1]:
                                    spanned = True
                                dataset.append((organism, normalize_labels(gene[0], region[0], exception), region[0], start, end, len(feature), spanned))

df = pd.DataFrame(dataset, columns=['organism', 'gene', 'region', 'start', 'end', 'length', 'spanned'])
display(df)

,organism,gene,region,start,end,length,spanned
0,Crepis callicephala,trnH-GUG,LSC,2,77,75,False
1,Crepis callicephala,psbA,LSC,474,1536,1062,False
2,Crepis callicephala,trnK,LSC,1758,4358,2600,False
3,Crepis callicephala,matK,LSC,2079,3591,1512,False
4,Crepis callicephala,trnK-UUU,LSC,4320,4357,37,False
...,...,...,...,...,...,...,...
266,Crepis purpurea,ycf2,IRA,140899,147754,6855,False
267,Crepis purpurea,trnM-CAU,IRA,147865,147940,75,False
268,Crepis purpurea,rpl23,IRA,148104,148386,282,False
269,Crepis purpurea,rpl2,IRA,148404,149894,1490,False


### merge synonymous labeles
#### Checking label to merge

In [7]:
organisms = df['organism'].unique()

differed = df.drop_duplicates(subset=['gene', 'region'], keep=False)
display(differed)

display(df[df['gene'].str.slice(0,4) == 'trnK'])
display(df[df['gene'].str.slice(0,4) == 'trnM'])
display(df[df['gene'].str.slice(0,4) == 'trnS'])
display(df[df['gene'].str.slice(0,4) == 'trnG'])
display(df[df['start'].between(29800, 34000)])

,organism,gene,region,start,end,length,spanned
2,Crepis callicephala,trnK,LSC,1758,4358,2600,False
25,Crepis callicephala,trnG-UCC,LSC,30094,30875,781,False
88,Crepis callicephala,trnM,IRB,83887,83961,74,False
132,Crepis callicephala,trnM,IRA,147833,147908,75,False
160,Crepis purpurea,trnS-CGA,LSC,30129,30911,782,False
223,Crepis purpurea,trnM-CAU,IRB,83900,83975,75,False
267,Crepis purpurea,trnM-CAU,IRA,147865,147940,75,False


,organism,gene,region,start,end,length,spanned
2,Crepis callicephala,trnK,LSC,1758,4358,2600,False
4,Crepis callicephala,trnK-UUU,LSC,4320,4357,37,False
138,Crepis purpurea,trnK-UUU,LSC,1760,4360,2600,False


,organism,gene,region,start,end,length,spanned
32,Crepis callicephala,trnM-CAU,LSC,36111,36185,74,False
47,Crepis callicephala,trnM-CAU,LSC,51913,51986,73,False
88,Crepis callicephala,trnM,IRB,83887,83961,74,False
132,Crepis callicephala,trnM,IRA,147833,147908,75,False
167,Crepis purpurea,trnM-CAU,LSC,36147,36221,74,False
182,Crepis purpurea,trnM-CAU,LSC,51948,52021,73,False
223,Crepis purpurea,trnM-CAU,IRB,83900,83975,75,False
267,Crepis purpurea,trnM-CAU,IRA,147865,147940,75,False


,organism,gene,region,start,end,length,spanned
9,Crepis callicephala,trnS-GCU,LSC,8489,8577,88,False
29,Crepis callicephala,trnS-UGA,LSC,34946,35029,83,False
37,Crepis callicephala,trnS-GGA,LSC,44777,44864,87,False
144,Crepis purpurea,trnS-GCU,LSC,8489,8577,88,False
160,Crepis purpurea,trnS-CGA,LSC,30129,30911,782,False
164,Crepis purpurea,trnS-UGA,LSC,34982,35065,83,False
172,Crepis purpurea,trnS-GGA,LSC,44812,44899,87,False


,organism,gene,region,start,end,length,spanned
25,Crepis callicephala,trnG-UCC,LSC,30094,30875,781,False
31,Crepis callicephala,trnG-GCC,LSC,35854,35925,71,False
166,Crepis purpurea,trnG-GCC,LSC,35890,35962,72,False


,organism,gene,region,start,end,length,spanned
24,Crepis callicephala,trnR-UCU,LSC,29817,29889,72,False
25,Crepis callicephala,trnG-UCC,LSC,30094,30875,781,False
26,Crepis callicephala,trnT-GGU,LSC,31010,31082,72,False
27,Crepis callicephala,psbD,LSC,32293,33355,1062,False
28,Crepis callicephala,psbC,LSC,33302,34724,1422,False
159,Crepis purpurea,trnR-UCU,LSC,29853,29925,72,False
160,Crepis purpurea,trnS-CGA,LSC,30129,30911,782,False
161,Crepis purpurea,trnT-GGU,LSC,31046,31118,72,False
162,Crepis purpurea,psbD,LSC,32329,33391,1062,False
163,Crepis purpurea,psbC,LSC,33338,34760,1422,False


#### Conclusion:
- 'trnK'-case: the additional instance of the trnK-UUU is due to the mistake on annottation cleaning - an alternative tRNA gene annotation was left in one of the exones. It should be removed manually or programmatically.
- 'trnG-trnS'-case reflects the annotation problem of the same sequence. Which is correct is difficult to judge.
- 'trnM'-case: trnM within the IR regions should be renamed to 'trnM-CAU'.

#### Another approach (nighbor matching)

Tested, but the problem seems to be easily resolved by only two corrections.

In [26]:

print(organisms)

def get_neighbours(index: int, df):
        idx = df.index.get_loc(index)
        if idx > 0:
            prev_gene = df.iloc[idx - 1]['gene']
        else:
            prev_gene = None
        if idx < len(df) - 1:
            next_gene = df.iloc[idx + 1]['gene']
        else:
            next_gene = None
        return prev_gene, next_gene

neighbours = {}

for i in df.index:
    
    gene = df.iloc[i, 1]
    print(i, gene)
    #print(type(i))
    if gene.startswith("trn"):
         prev, next = get_neighbours(i, df)
         print(prev, next)

# NB: There are problems in the code, look at the output for idx = 7 (psbK)
#
# It is OK, look at the `if gene.startswith('trn')` line

['Crepis callicephala' 'Crepis purpurea']
0 trnH-GUG
None psbA
1 psbA
2 trnK
psbA matK
3 matK
4 trnK-UUU
matK rps16
5 rps16
6 trnQ-UUG
rps16 psbK
7 psbK
8 psbI
9 trnS-GCU
psbI trnC-GCA
10 trnC-GCA
trnS-GCU petN
11 petN
12 psbM
13 trnD-GUC
psbM trnY-GUA
14 trnY-GUA
trnD-GUC trnE-UUC
15 trnE-UUC
trnY-GUA rpoB
16 rpoB
17 rpoC1
18 rpoC2
19 rps2
20 atpI
21 atpH
22 atpF
23 atpA
24 trnR-UCU
atpA trnG-UCC
25 trnG-UCC
trnR-UCU trnT-GGU
26 trnT-GGU
trnG-UCC psbD
27 psbD
28 psbC
29 trnS-UGA
psbC psbZ
30 psbZ
31 trnG-GCC
psbZ trnM-CAU
32 trnM-CAU
trnG-GCC rps14
33 rps14
34 psaB
35 psaA
36 pafI
37 trnS-GGA
pafI rps4
38 rps4
39 trnT-UGU
rps4 trnL-UAA
40 trnL-UAA
trnT-UGU trnF-GAA
41 trnF-GAA
trnL-UAA ndhJ
42 ndhJ
43 ndhK
44 psbG
45 ndhC
46 trnV-UAC
ndhC trnM-CAU
47 trnM-CAU
trnV-UAC atpE
48 atpE
49 atpB
50 rbcL
51 accD
52 psaI
53 pafII
54 cemA
55 petA
56 psbJ
57 psbL
58 psbF
59 psbE
60 petL
61 petG
62 trnW-CCA
petG trnP-UGG
63 trnP-UGG
trnW-CCA psaJ
64 psaJ
65 rpl33
66 rps18
67 rpl20
68 rps12
69 clp

In [107]:
df_repeats = df[df['region'].isin(['IRA', 'IRB'])]

pivot_table = df_repeats.pivot_table(
    values='length',
    index='gene',
    columns=['organism', 'region'],
    aggfunc='first'  # or 'mean', 'sum', etc. depending on your needs
)

# Reorder levels in the columns
#pivot_table = df2.reorder_levels([1, 0], axis=1)

# Sort the columns
pivot_table = pivot_table.sort_index(axis=1)

# Display the pivot table
display(pivot_table)


organism       Crepis callicephala         Crepis purpurea        
region                         IRA     IRB             IRA     IRB
gene                                                              
ndhB                        2202.0  2202.0          2202.0  2202.0
ndhF                           NaN  2226.0             NaN  2226.0
rpl2                        1490.0  1490.0          1490.0  1490.0
rpl23                        282.0   282.0           282.0   282.0
rps19 fragment                60.0     NaN            55.0     NaN
rps7                         468.0   468.0           468.0   468.0
rrn16                       1490.0  1490.0          1490.0  1490.0
rrn23                       2810.0  2810.0          2810.0  2810.0
rrn4.5                       103.0   103.0           103.0   103.0
rrn5                         122.0   116.0           121.0   122.0
trnA-UGC                     893.0   893.0           893.0   893.0
trnI-GAU                     834.0   834.0           836.0   836.0
trnL-CAA                      81.0    81.0            83.0    83.0
trnM                          75.0    74.0             NaN     NaN
trnM-CAU                       NaN     NaN            75.0    75.0
trnN-GUU                      72.0    72.0            74.0    74.0
trnR-ACG                      74.0    74.0            75.0    75.0
trnV-GAC                      73.0    72.0            72.0    72.0
ycf1 fragment                  NaN   480.0             NaN   480.0
ycf15                        192.0   192.0           192.0   192.0
ycf2                        6855.0  6855.0          6855.0  6855.0

Checking ycf1 to develope the logic for renaming

In [27]:
import pandas as pd
import numpy as np
from Bio import SeqRecord, SeqIO, SeqFeature

# variables
c_cal = "plastomes/in/C_callicephala_curated_1_cleaned_meta.gb"
c_pur = "plastomes/in/Crepis_purpurea_edited+.gb"

regions = {}
dataset = []

# Do initial parsing
for acc in c_cal, c_pur:
    with open(acc, "r") as handle:
        for record in SeqIO.parse(handle, "genbank"):
            features = record.features
            # check if there features other than 'source'
            if len(features) > 1:
                for feature in features:
                    # check for genome regions
                    if feature.type in ['misc_feature', 'repeat_region']:
                        label = feature.qualifiers.get('note', "")[0].split()[-1].strip("()")
                        regions[label] = [int(feature.location.start), int(feature.location.end)]
                # filling the dataframe
                organism = record.annotations.get('organism', "")
                for feature in features:
                    if feature.type == 'gene':
                        gene = feature.qualifiers.get('gene', "")
                        if "ycf1" in gene[0] and not "5" in gene[0]:
                            start = int(feature.location.start)
                            end = int(feature.location.end)

                            for region in regions.items():
                                ranges = region[-1]
                                if ranges[0] <= start < ranges[-1]:
                                    #print(f'{region[0]}\t{gene}')
                                    spanned = False
                                    if end >= ranges[-1]:
                                        spanned = True
                                    dataset.append((organism, gene[0], region[0], start, end, len(feature), spanned))

df = pd.DataFrame(dataset, columns=['organism', 'gene', 'region', 'start', 'end', 'length', 'spanned'])
display(df)

,organism,gene,region,start,end,length,spanned
0,Crepis callicephala,ycf1,IRB,106267,106747,480,True
1,Crepis callicephala,ycf1,SSC,120557,125527,4970,True
2,Crepis purpurea,ycf1 fragment,IRB,106289,106769,480,True
3,Crepis purpurea,ycf1,SSC,120578,125551,4973,True


checking trna fragmentation in inverted repeats: case of trnL-CAA fragment

In [28]:
import pandas as pd
import numpy as np
from Bio import SeqRecord, SeqIO, SeqFeature

# variables
c_cal = "plastomes/in/C_callicephala_curated_1_cleaned_meta.gb"
c_pur = "plastomes/in/Crepis_purpurea_edited+.gb"

regions = {}
dataset = []

# Do initial parsing
for acc in c_cal, c_pur:
    with open(acc, "r") as handle:
        for record in SeqIO.parse(handle, "genbank"):
            features = record.features
            # check if there features other than 'source'
            if len(features) > 1:
                for feature in features:
                    # check for genome regions
                    if feature.type in ['misc_feature', 'repeat_region']:
                        label = feature.qualifiers.get('note', "")[0].split()[-1].strip("()")
                        regions[label] = [int(feature.location.start), int(feature.location.end)]
                # filling the dataframe
                organism = record.annotations.get('organism', "")
                for feature in features:
                    if feature.type == 'gene':
                        gene = feature.qualifiers.get('gene', "")
                        if "trnL" in gene[0]:
                            start = int(feature.location.start)
                            end = int(feature.location.end)

                            for region in regions.items():
                                ranges = region[-1]
                                if ranges[0] <= start < ranges[-1]:
                                    #print(f'{region[0]}\t{gene}')
                                    spanned = False
                                    if end >= ranges[-1]:
                                        spanned = True
                                    dataset.append((organism, gene[0], region[0], start, end, len(feature), spanned))

df = pd.DataFrame(dataset, columns=['organism', 'gene', 'region', 'start', 'end', 'length', 'spanned'])
display(df)

,organism,gene,region,start,end,length,spanned
0,Crepis callicephala,trnL-UAA,LSC,46769,47293,524,False
1,Crepis callicephala,trnL-CAA,IRB,91329,91410,81,False
2,Crepis callicephala,trnL-UAG,SSC,110966,111046,80,False
3,Crepis callicephala,trnL-CAA,IRA,140384,140465,81,False
4,Crepis purpurea,trnL-UAA | trnL,LSC,46804,47329,525,False
5,Crepis purpurea,trnL-CAA | trnL-fragment,IRB,91342,91425,83,False
6,Crepis purpurea,trnL-UAG | trnL,SSC,110983,111063,80,False
7,Crepis purpurea,trnL-CAA | trnL-fragment,IRA,140415,140498,83,False


Compare features within LSC after labels being normalized

In [30]:
df_target = df[df['region'].isin(['LSC'])]

non_tRNA = df_target[~df_target['gene'].str.startswith('trn')]
tRNA = df_target[df_target['gene'].str.startswith('trn')]

pt_tRNA = tRNA.pivot_table(
    values='length',
    index='gene',
    columns=['organism'],
    aggfunc='first'  # or 'mean', 'sum', etc. depending on your needs
)

# Sort the columns
pt_tRNA = pt_tRNA.sort_index(axis=1)

pt_nontRNA = non_tRNA.pivot_table(
    values='length',
    index='gene',
    columns=['organism'],
    aggfunc='first'  # or 'mean', 'sum', etc. depending on your needs
)

# Sort the columns
pt_nontRNA = pt_nontRNA.sort_index(axis=1)


# Display the pivot table
#display(pt_tRNA, pt_nontRNA)
#with pd.option_context('display.max_rows', None,):
#    display(pt_nontRNA)

# Find differences
diff_nontRNA = pt_nontRNA['Crepis callicephala'] != pt_nontRNA['Crepis purpurea']
print("Genes with different values:")
display(pt_nontRNA[diff_nontRNA])

diff_tRNA = pt_tRNA['Crepis callicephala'] != pt_tRNA['Crepis purpurea']
print("tRNA genes with different values:")
display(pt_tRNA[diff_tRNA])

Genes with different values:


organism,Crepis callicephala,Crepis purpurea
gene,,
petD,525,1183
psaA,2253,1776
rpl16,1445,1440
rpl22,465,540
rpoC1,2799,2520
rpoC2,4125,2274
rps16,1120,1125


tRNA genes with different values:


organism,Crepis callicephala,Crepis purpurea
gene,,
trnD-GUC,74.0,77.0
trnE-UUC,72.0,74.0
trnF-GAA,70.0,74.0
trnG-GCC,71.0,72.0
trnG-UCC,781.0,NaN
trnK,2600.0,NaN
trnK-UUU,37.0,2600.0
trnL-UAA,524.0,525.0
trnP-UGG,75.0,76.0


Compare features which labels where normalized, within SSC region.

In [109]:
df_target = df[df['region'].isin(['SSC'])]

non_tRNA = df_target[~df_target['gene'].str.startswith('trn')]
tRNA = df_target[df_target['gene'].str.startswith('trn')]

pt_tRNA = tRNA.pivot_table(
    values='length',
    index='gene',
    columns=['organism'],
    aggfunc='first'  # or 'mean', 'sum', etc. depending on your needs
)

# Sort the columns
pt_tRNA = pt_tRNA.sort_index(axis=1)

pt_nontRNA = non_tRNA.pivot_table(
    values='length',
    index='gene',
    columns=['organism'],
    aggfunc='first'  # or 'mean', 'sum', etc. depending on your needs
)

# Sort the columns
pt_nontRNA = pt_nontRNA.sort_index(axis=1)


# Display the pivot table
#display(pt_tRNA, pt_nontRNA)
#with pd.option_context('display.max_rows', None,):
#    display(pt_nontRNA)

# Find differences
diff_nontRNA = pt_nontRNA['Crepis callicephala'] != pt_nontRNA['Crepis purpurea']
print("Genes with different values:")
display(pt_nontRNA[diff_nontRNA])

diff_tRNA = pt_tRNA['Crepis callicephala'] != pt_tRNA['Crepis purpurea']
print("tRNA genes with different values:")
display(pt_tRNA[diff_tRNA])

Genes with different values:


organism,Crepis callicephala,Crepis purpurea
gene,,
ndhA,2147,2146
ndhD,1503,1521
ycf1,4970,4973


tRNA genes with different values:


organism,Crepis callicephala,Crepis purpurea
gene,,


### Applying changes

In [8]:
import pandas as pd
import numpy as np
from Bio import SeqRecord, SeqIO, SeqFeature

# variables
c_cal = "plastomes/in/C_callicephala_curated_1_cleaned_meta.gb"
c_pur = "plastomes/in/Crepis_purpurea_edited+.gb"

exception = [
    "trnL-CAA fragment",
    "trnT-GGU | trnT-fragment",
]

regions = {}
dataset = []
new_record = []

# Do initial parsing
for acc in c_cal, c_pur:
    print(acc)
    output = acc.replace("/in/", "/out/")
    output = output.replace("C_callicephala_curated_1_cleaned_meta", "Crepis_callicephala")
    output = output.replace("Crepis_purpurea_edited+", "Crepis_purpurea")
    print(acc, output)

    new_features = []

    with open(acc, "r") as handle:
        for record in SeqIO.parse(handle, "genbank"):
            features = record.features
            # check if there features other than 'source'
            if len(features) > 1:
                for feature in features:
                    # check for genome regions
                    if feature.type in ['misc_feature', 'repeat_region']:
                        label = feature.qualifiers.get('note', "")[0].split()[-1].strip("()")
                        regions[label] = [int(feature.location.start), int(feature.location.end)]
                # filling the dataframe
                organism = record.annotations.get('organism', "")
                for feature in features:
                    if feature.type == 'gene':
                        
                        gene = feature.qualifiers.get('gene', "")
                        start = int(feature.location.start)
                        end = int(feature.location.end)

                        # handling 'trnM' case
                        if gene[0] == "trnM":
                            gene[0] = "trnM-CAU"

                        # renaming 'trnK'
                        if gene[0] == "trnK":
                            gene[0] = "trnK-UUU"

                        # handling the case of excessive trnK
                        if gene[0] == "trnK-UUU" and len(feature) < 100:
                            pass
                        else:
                            for region in regions.items():
                                gene_label = normalize_labels(gene[0], region[0], exception)

                                feature.qualifiers['gene'] = gene_label

                                ranges = region[-1]
                                if ranges[0] <= start < ranges[-1]:
                                    #print(f'{region[0]}\t{gene}')
                                    spanned = False
                                    if end >= ranges[-1]:
                                        spanned = True
                                    dataset.append((organism, normalize_labels(gene[0], region[0], exception), region[0], start, end, len(feature), spanned))

                    new_features.append(feature)
                    new_record = SeqRecord.SeqRecord(
                        seq=record.seq,
                        id=record.id,
                        name=record.name,
                        description=record.description,
                        features=new_features,
                        annotations=record.annotations
                    )

                    print("features:", len(features), type(features))
                    print("records:", len(record), type(record))
                    SeqIO.write(new_record, output, "genbank")

df = pd.DataFrame(dataset, columns=['organism', 'gene', 'region', 'start', 'end', 'length', 'spanned'])
display(df)
display(df[df['gene'].str.slice(0,4) == 'trnM'])
display(df[df['gene'].str.slice(0,4) == 'trnK'])




plastomes/in/C_callicephala_curated_1_cleaned_meta.gb
plastomes/in/C_callicephala_curated_1_cleaned_meta.gb plastomes/out/Crepis_callicephala.gb
features: 348 <class 'list'>
records: 149980 <class 'Bio.SeqRecord.SeqRecord'>
features: 348 <class 'list'>
records: 149980 <class 'Bio.SeqRecord.SeqRecord'>
features: 348 <class 'list'>
records: 149980 <class 'Bio.SeqRecord.SeqRecord'>
features: 348 <class 'list'>
records: 149980 <class 'Bio.SeqRecord.SeqRecord'>
features: 348 <class 'list'>
records: 149980 <class 'Bio.SeqRecord.SeqRecord'>
features: 348 <class 'list'>
records: 149980 <class 'Bio.SeqRecord.SeqRecord'>
features: 348 <class 'list'>
records: 149980 <class 'Bio.SeqRecord.SeqRecord'>
features: 348 <class 'list'>
records: 149980 <class 'Bio.SeqRecord.SeqRecord'>
features: 348 <class 'list'>
records: 149980 <class 'Bio.SeqRecord.SeqRecord'>
features: 348 <class 'list'>
records: 149980 <class 'Bio.SeqRecord.SeqRecord'>
features: 348 <class 'list'>
records: 149980 <class 'Bio.SeqRecor

,organism,gene,region,start,end,length,spanned
0,Crepis callicephala,trnH-GUG,LSC,2,77,75,False
1,Crepis callicephala,psbA,LSC,474,1536,1062,False
2,Crepis callicephala,trnK-UUU,LSC,1758,4358,2600,False
3,Crepis callicephala,matK,LSC,2079,3591,1512,False
4,Crepis callicephala,rps16,LSC,5149,6269,1120,False
...,...,...,...,...,...,...,...
265,Crepis purpurea,ycf2,IRA,140899,147754,6855,False
266,Crepis purpurea,trnM-CAU,IRA,147865,147940,75,False
267,Crepis purpurea,rpl23,IRA,148104,148386,282,False
268,Crepis purpurea,rpl2,IRA,148404,149894,1490,False


,organism,gene,region,start,end,length,spanned
31,Crepis callicephala,trnM-CAU,LSC,36111,36185,74,False
46,Crepis callicephala,trnM-CAU,LSC,51913,51986,73,False
87,Crepis callicephala,trnM-CAU,IRB,83887,83961,74,False
131,Crepis callicephala,trnM-CAU,IRA,147833,147908,75,False
166,Crepis purpurea,trnM-CAU,LSC,36147,36221,74,False
181,Crepis purpurea,trnM-CAU,LSC,51948,52021,73,False
222,Crepis purpurea,trnM-CAU,IRB,83900,83975,75,False
266,Crepis purpurea,trnM-CAU,IRA,147865,147940,75,False


,organism,gene,region,start,end,length,spanned
2,Crepis callicephala,trnK-UUU,LSC,1758,4358,2600,False
137,Crepis purpurea,trnK-UUU,LSC,1760,4360,2600,False


## Changing Reference field in genbank file.

In [ ]:
import pandas as pd
import numpy as np
from Bio import SeqRecord, SeqIO, SeqFeature
import os

# variables
c_cal = "plastomes/out/Crepis_callicephala.gb"
c_pur = "plastomes/out/Crepis_purpurea.gb"
cich = "plastomes/test/cichorieae_seqrecords.gb"


# Do initial parsing
for acc in c_cal, c_pur:
    print(acc)
    
    with open(acc, "r") as handle:
        for record in SeqIO.parse(handle, "genbank"):

            # Deleye Comment field if present
            if 'comment' in record.annotations.keys():
                record.annotations.pop('comment')
            
            refs = record.annotations.get('references', [])
            print(record.annotations.get('organism', ""), record.id)
            print(f'initial refs length: {len(refs)}')

            authors = "Asan O. Emirsaliev, Dmitry A. Afonnikov, Irina V. Mitrofanova, Elena A. Salina, Alex V. Kochetov"
            title = "Phylogenetic analysis of rare Crepis species of Crimean flora (C. purpurea (Willd.) M. Bieb. and C. callicephala Juz.)"
            journal: "Unpublished"
            department = "Department of Plant Molecular Genetics, IC&G SB RAS. 10, Ac. Lavrentieva ave., Novosibirsk, 630090, Russia"

            for ref in refs:
                #print(type(refs), type(ref), ref)
                #print(ref)
                if ref.title == "GeSeq - versatile and accurate annotation of organelle genomes":
                    removed = refs.pop()
                    #print("removed:", removed)
            
            new_ref = SeqFeature.Reference()
            new_ref.authors = authors
            new_ref.title = title
            new_ref.journal = "Unpublished"
            
            submit_ref = SeqFeature.Reference()
            submit_ref.authors = authors
            submit_ref.title = "Direct Submission"
            submit_ref.journal = department

            record.annotations['references'] = [new_ref, submit_ref]
            #print(f'\nREFS:')
            for ref in refs:
                #print(ref)
                pass

            output = acc.replace("/out/", "/w-refs/")
            print(output)

            dir_path = os.path.dirname(output)
            print(dir_path)
            os.makedirs(dir_path, exist_ok=True)
            SeqIO.write(record, output, "genbank")

            print(f'final refs length: {len(refs)}')

plastomes/out/Crepis_callicephala.gb
Crepis callicephala Crepis_callicephala
initial refs length: 0
plastomes/w-refs/Crepis_callicephala.gb
plastomes/w-refs
final refs length: 0
plastomes/out/Crepis_purpurea.gb
Crepis purpurea Crepis_purpurea
initial refs length: 1
plastomes/w-refs/Crepis_purpurea.gb
plastomes/w-refs
final refs length: 0


### Preparing gb file for submission.
Sequin is not working, and GB2Sequin tool only modifies Sequin files, but not tab-delimited files.

In [26]:
import pandas as pd
import numpy as np
from Bio import SeqRecord, SeqIO, SeqFeature
import os

# variables
c_cal = "plastomes/w-refs/Crepis_callicephala.gb"
c_pur = "plastomes/w-refs/Crepis_purpurea.gb"
cich = "plastomes/test/cichorieae_seqrecords.gb"


# Do initial parsing
for acc in [c_cal, c_pur]:
    print(acc)
    
    with open(acc, "r") as handle:
        for record in SeqIO.parse(handle, "genbank"):
            if not "angustana" in record.annotations.get('organism', ""):
                features = record.features
                print(record.id, record.annotations.get('organism', ""), len(features) - 1)
                
                # check if there are features other than 'source'
                if len(features) > 1:
                    for feature in features:

                        # remove 'annotator' and 'info' keys that are not valid
                        if "annotator" in feature.qualifiers.keys():
                            feature.qualifiers.pop('annotator')
                        if "info" in feature.qualifiers.keys():
                            feature.qualifiers.pop('info')

                        # fix 'trans_splicin' error in both genes and CDSs
                        if feature.qualifiers.get('gene', [""])[0] == "rps12":
                                    if 'trans_splicin' in feature.qualifiers.keys():
                                        val = feature.qualifiers.get('trans_splicin', "")
                                        feature.qualifiers.pop('trans_splicin')
                                        feature.qualifiers['trans_splicing'] = val
                                    #print(feature)

                        t = feature.type
                        match t:
                            case "CDS":
                                #print(t, feature.qualifiers)
                                if not 'product' in feature.qualifiers.keys():
                                    #print(f'BEFORE fixing:\n{feature}')
                                    gene_to_look = feature.qualifiers.get('gene', [""])[0]
                                    match gene_to_look:
                                        case "ycf1":
                                            feature.qualifiers['product'] = "Putative membrane protein ycf1; RF1; hypothetical chloroplast RF1 (chloroplast)"
                                        case "psbG":
                                            feature.qualifiers['product'] = "PsbG; Photosystem II G protein"
                                        case "accD":
                                            feature.qualifiers['product'] = "Acetyl-coenzyme A carboxylase carboxyl transferase subunit beta, chloroplastic; ACCase subunit beta; Acetyl-CoA carboxylase carboxyltransferase subunit beta; AccD"
                                        case "rps19-fragment":
                                            feature.qualifiers['product'] = "Small ribosomal subunit protein uS19c; 30S ribosomal protein S19, chloroplastic; Ribosomal protein S19"
                                    #print(f'AFTER fixing:\n{feature}')
                                    pass
                                # fixing translation table issue
                                if "transl_table" in feature.qualifiers.keys():
                                    #print(feature.qualifiers['transl_table'])
                                    pass
                                else:
                                    feature.qualifiers['transl_table'] = 11
                            case "gene":
                                pass

                output = acc.replace("w-refs", "final")
                print(output)

                dir_path = os.path.dirname(output)
                print(dir_path)
                os.makedirs(dir_path, exist_ok=True)
                SeqIO.write(record, output, "genbank")
                        

plastomes/w-refs/Crepis_callicephala.gb
Crepis_callicephala Crepis callicephala 347
plastomes/final/Crepis_callicephala.gb
plastomes/final
plastomes/w-refs/Crepis_purpurea.gb
Crepis_purpurea Crepis purpurea 352
plastomes/final/Crepis_purpurea.gb
plastomes/final


### List of errors in annotation file processing

1. ERROR: valid [SEQ_FEAT.InternalStop] 1 internal stops. Genetic code [11] FEATURE: CDS: PsbG; Photosystem II G protein [lcl|Crepis_callicephala:c49645-49597] [lcl|Crepis_callicephala: raw, dna len= 149980] -> [lcl|Crepis_callicephala_26]

**DESCRIPTION:** The CDS entity has no annotation. The possible solution would be checking if CDS would have translation, and add adding to the resultant annotation.

2. ERROR: valid [SEQ_FEAT.NoStop] Missing stop codon FEATURE: CDS: PsbG; Photosystem II G protein [lcl|Crepis_callicephala:c49645-49597] [lcl|Crepis_callicephala: raw, dna len= 149980] -> [lcl|Crepis_callicephala_26]

**DESCRIPTION:** There are no stop codon within expected length of the gene. There are several possibilities for optimal solution:
- pseudogenize the gene.
- expand untill the stop codon.
- break on internal stop codon if exist.

In [10]:
import pandas as pd
import numpy as np
from Bio import SeqRecord, SeqIO
from Bio.SeqFeature import SeqFeature, SimpleLocation
import os

# variables
c_cal = "plastomes/final/Crepis_callicephala.gb"
c_pur = "plastomes/final/Crepis_purpurea.gb"
cich = "plastomes/test/cichorieae_seqrecords.gb"


# Do initial parsing
for acc in [c_cal]:
    print(acc)
    
    with open(acc, "r") as handle:
        for record in SeqIO.parse(handle, "genbank"):
            if not "angustana" in record.annotations.get('organism', ""):
                features = record.features
                print(record.id, record.annotations.get('organism', ""), len(features) - 1)
                
                # check if there are features other than 'source'
                if len(features) > 1:
                    for feature in features:
                        if feature.type == "CDS" and feature.qualifiers.get('gene', [""])[0] == "psbG":
                            print(feature)
                            a = feature.extract(record)
                            print(a)
                            b = SeqFeature(SimpleLocation(49591, 49645, strand=-1), type="CDS")
                            print(b.extract(record))
                            ls = "ATGGTCTTAGCTCCTGAATATTCAGACAATAAAAAGAAAAAAGGAAAAAATTGA"
                            print(ls)
                            print(f'L. sativa: {len(ls)}\nC.callicephala: {len(b)}')
                            c = b.translate(record, table=11)
                            print(c)

plastomes/final/Crepis_callicephala.gb
Crepis_callicephala Crepis callicephala 347
type: CDS
location: [49596:49645](-)
qualifiers:
    Key: gene, Value: ['psbG']
    Key: product, Value: ['PsbG; Photosystem II G protein']
    Key: transl_table, Value: ['11']

ID: <unknown id>
Name: <unknown name>
Description: <unknown description>
Number of features: 2
Seq('ATGGTCTTAGCTCCTGAATATTAAGACAATAAAAAGAAAAAAGGAAAAA')
ID: <unknown id>
Name: <unknown name>
Description: <unknown description>
Number of features: 2
Seq('ATGGTCTTAGCTCCTGAATATTAAGACAATAAAAAGAAAAAAGGAAAAAATTTA')
ATGGTCTTAGCTCCTGAATATTCAGACAATAAAAAGAAAAAAGGAAAAAATTGA
L. sativa: 54
C.callicephala: 54


TranslationError: Final codon 'TTA' is not a stop codon

Aligning protein PsbG against plastid genome sequence.

In [2]:
import os

pept = "plastomes/refs/psbG_A0A5C0PXP7.fasta"
ref = "plastomes/final/gb2sequin_out/c_callicephala_old/Crepis_callicephala___149980_bp____DNA.fsa.txt"

cmd = f'bash aln_aa2na.sh -f {pept} -r {ref} -o output.txt -t 2'

import subprocess

subprocess.run(['bash', 'aln_aa2na.sh', '-f', pept, 
                '-r', ref, 
                '-o', 'aln_PsbG_2_cc.txt', '-t', '2'], check=True)

CompletedProcess(args=['bash', 'aln_aa2na.sh', '-f', 'plastomes/refs/psbG_A0A5C0PXP7.fasta', '-r', 'plastomes/final/gb2sequin_out/c_callicephala_old/Crepis_callicephala___149980_bp____DNA.fsa.txt', '-o', 'aln_PsbG_2_cc.txt', '-t', '2'], returncode=0)

Successfully aligned 10 aa with only mismatch in range of 49645..49616 (reverse strand).

Aligning genic na sequence against plastome.

In [5]:
gene = "plastomes/refs/psbG_lac.fna"
ref = "plastomes/final/gb2sequin_out/c_callicephala_old/Crepis_callicephala___149980_bp____DNA.fsa.txt"

import subprocess

subprocess.run(['bash', 'aln_na2na.sh', '-f', gene, 
                '-r', ref, 
                '-o', 'aln_psbG-lac_2_cc.txt', '-t', '2'], check=True)

CompletedProcess(args=['bash', 'aln_na2na.sh', '-f', 'plastomes/refs/psbG_lac.fna', '-r', 'plastomes/final/gb2sequin_out/c_callicephala_old/Crepis_callicephala___149980_bp____DNA.fsa.txt', '-o', 'aln_psbG-lac_2_cc.txt', '-t', '2'], returncode=0)

У Lactuca sativa ген на 5'-конце имеет почти ту же последовательность (с одной заменой), что и у Crepis callicephala. Делеции, как в случае с Hypohaeris, нет. Доводим его до той же длины, что у L. sativa, редактируя границы сущности.

3. ERROR: valid [SEQ_FEAT.StartCodon] Illegal start codon (and 4 internal stops). Probably wrong genetic code [11] FEATURE: CDS: Acetyl-coenzyme A carboxylase carboxyl transferase subunit beta, chloroplastic; ACCase subunit beta; Acetyl-CoA carboxylase carboxyltransferase subunit beta; AccD [lcl|Crepis_callicephala:56772-57064] [lcl|Crepis_callicephala: raw, dna len= 149980] -> [lcl|Crepis_callicephala_31]

**DESCRIPTION:** the gene should be marked as pseudogene, and the CDS removed from the annotation. Qualiviers `pseudo` and `pseudogene=”unknown"` might be added to both gene and CDS features.

> Qualifier `pseudo` indicates that this feature is a non-functional version of the element named by the feature key.
>
>The qualifier `pseudo` should be used to describe non-functional 
>                 genes that are not formally described as pseudogenes, e.g. CDS 
>                 has no translation due to other reasons than pseudogenisation events.
>                 Other reasons may include sequencing or assembly errors.
>                 In order to annotate pseudogenes the qualifier /pseudogene= must be
>                 used indicating the TYPE which can be taken from the INSDC controlled vocabulary 
>                 for pseudogenes.

> Qualifier `pseudogene=` indicates that this feature is a pseudogene of the element named by the feature key. 
> where TYPE is one of the following: processed, unprocessed, unitary, allelic, unknown.
> 
> unknown: the submitter does not know the method of pseudogenisation.

4. ERROR: valid [SEQ_FEAT.InternalStop] 4 internal stops (and illegal start codon). Genetic code [11] FEATURE: CDS: Acetyl-coenzyme A carboxylase carboxyl transferase subunit beta, chloroplastic; ACCase subunit beta; Acetyl-CoA carboxylase carboxyltransferase subunit beta; AccD [lcl|Crepis_callicephala:56772-57064] [lcl|Crepis_callicephala: raw, dna len= 149980] -> [lcl|Crepis_callicephala_31]

5. ERROR: valid [SEQ_FEAT.NoStop] Missing stop codon FEATURE: CDS: Acetyl-coenzyme A carboxylase carboxyl transferase subunit beta, chloroplastic; ACCase subunit beta; Acetyl-CoA carboxylase carboxyltransferase subunit beta; AccD [lcl|Crepis_callicephala:56772-57064] [lcl|Crepis_callicephala: raw, dna len= 149980] -> [lcl|Crepis_callicephala_31]


In [11]:
import pandas as pd
import numpy as np
from Bio import SeqRecord, SeqIO
from Bio.SeqFeature import SeqFeature, SimpleLocation
import os

# variables
c_cal = "plastomes/final/Crepis_callicephala.gb"
c_pur = "plastomes/final/Crepis_purpurea.gb"
cich = "plastomes/test/cichorieae_seqrecords.gb"


# Do initial parsing
for acc in [c_cal]:
    print(acc)
    
    with open(acc, "r") as handle:
        for record in SeqIO.parse(handle, "genbank"):
            if not "angustana" in record.annotations.get('organism', ""):
                features = record.features
                print(record.id, record.annotations.get('organism', ""), len(features) - 1)
                
                # check if there are features other than 'source'
                if len(features) > 1:
                    for feature in features:
                        if feature.qualifiers.get('gene', [""])[0] == "accD":
                            print(feature)

plastomes/final/Crepis_callicephala.gb
Crepis_callicephala Crepis callicephala 347
type: gene
location: [56771:57064](+)
qualifiers:
    Key: gene, Value: ['accD']

type: CDS
location: [56771:57064](+)
qualifiers:
    Key: gene, Value: ['accD']
    Key: product, Value: ['Acetyl-coenzyme A carboxylase carboxyl transferase subunit beta, chloroplastic; ACCase subunit beta; Acetyl-CoA carboxylase carboxyltransferase subunit beta; AccD']
    Key: transl_table, Value: ['11']



6. ERROR: valid [SEQ_FEAT.StartCodon] Illegal start codon used. Wrong genetic code [11] or protein should be partial FEATURE: CDS: photosystem II subunit L [lcl|Crepis_callicephala:c61997-61881] [lcl|Crepis_callicephala: raw, dna len= 149980] -> [lcl|Crepis_callicephala_37]

**DESCRIPTION:** the psbL gene ORF starts in ACG codon. There is no option to point to the alternative start codon in genbank file.

7. ERROR: valid [SEQ_FEAT.StartCodon] Illegal start codon (and 6 internal stops). Probably wrong genetic code [11] FEATURE: CDS: ribosomal protein S12 [(lcl|Crepis_callicephala:c136794-136768, c136232-136002, c67482-67369)] [lcl|Crepis_callicephala: raw, dna len= 149980] -> [lcl|Crepis_callicephala_46]

**DESCRIPTION:** 

8. ERROR: valid [SEQ_FEAT.InternalStop] 6 internal stops (and illegal start codon). Genetic code [11] FEATURE: CDS: ribosomal protein S12 [(lcl|Crepis_callicephala:c136794-136768, c136232-136002, c67482-67369)] [lcl|Crepis_callicephala: raw, dna len= 149980] -> [lcl|Crepis_callicephala_46]

**DESCRIPTION:** 

In [15]:
import pandas as pd
import numpy as np
from Bio import SeqRecord, SeqIO
from Bio.SeqFeature import SeqFeature, SimpleLocation
import os

# variables
c_cal = "plastomes/final/Crepis_callicephala.gb"
c_pur = "plastomes/final/Crepis_purpurea.gb"
cich = "plastomes/test/cichorieae_seqrecords.gb"


# Do initial parsing
for acc in [c_cal]:
    print(acc)
    
    with open(acc, "r") as handle:
        for record in SeqIO.parse(handle, "genbank"):
            if not "angustana" in record.annotations.get('organism', ""):
                features = record.features
                print(record.id, record.annotations.get('organism', ""), len(features) - 1)
                
                # check if there are features other than 'source'
                if len(features) > 1:
                    for feature in features:
                        if feature.qualifiers.get('gene', [""])[0] == "psbL":
                            print(feature.location)
                            #print(feature.translate(record, table=11))
                            print(feature.extract(record))

plastomes/final/Crepis_callicephala.gb
Crepis_callicephala Crepis callicephala 347
[61880:61997](-)
ID: <unknown id>
Name: <unknown name>
Description: <unknown description>
Number of features: 2
Seq('ACGACACAATCAAACCCAAACGAACAAAATGTTGAATTGAATCGTACCAGCCTC...TAA')
[61880:61997](-)
ID: <unknown id>
Name: <unknown name>
Description: <unknown description>
Number of features: 2
Seq('ACGACACAATCAAACCCAAACGAACAAAATGTTGAATTGAATCGTACCAGCCTC...TAA')


In [20]:
import pandas as pd
import numpy as np
from Bio import SeqRecord, SeqIO
from Bio.SeqFeature import SeqFeature, SimpleLocation
import os

# variables
c_cal = "plastomes/final/Crepis_callicephala.gb"
c_pur = "plastomes/final/Crepis_purpurea.gb"
cich = "plastomes/test/cichorieae_seqrecords.gb"


# Do initial parsing
for acc in [c_cal]:
    print(acc)
    
    with open(acc, "r") as handle:
        for record in SeqIO.parse(handle, "genbank"):
            if not "angustana" in record.annotations.get('organism', ""):
                features = record.features
                print("\n", record.id, record.annotations.get('organism', ""), len(features) - 1)
                
                # check if there are features other than 'source'
                if len(features) > 1:
                    for feature in features:
                        if feature.qualifiers.get('gene', [""])[0] == "rps12":
                            #print(f'{feature.location}')
                            #print(feature.translate(record, table=11))
                            ex = feature.extract(record)
                            print(f'\n{feature.type}\n{feature.location}\n{ex}')

plastomes/final/Crepis_callicephala.gb

 Crepis_callicephala Crepis callicephala 347

gene
join{[136001:136794](-), [67368:67482](-)}
ID: <unknown id>
Name: <unknown name>
Description: <unknown description>
Number of features: 4
Seq('TTATTTTGGCTTTTTTACCCCATATTGTAGGGTGGATCTCGAAAGATATGAAAG...TAT')

CDS
join{[136767:136794](-), [136001:136232](-), [67368:67482](-)}
ID: <unknown id>
Name: <unknown name>
Description: <unknown description>
Number of features: 3
Seq('TTATTTTGGCTTTTTTACCCCATATTGAGAACGCCCTTGTTGACGATCCTTTAC...TAT')

exon
[67368:67482](-)
ID: <unknown id>
Name: <unknown name>
Description: <unknown description>
Number of features: 1
Seq('ATGCCAACGATTAAACAACTTATTAGAAATACAAGACAGCCAATTAGAAATGTC...TAT')

gene
join{[67368:67482](-), [95000:95793](-)}
ID: <unknown id>
Name: <unknown name>
Description: <unknown description>
Number of features: 4
Seq('ATGCCAACGATTAAACAACTTATTAGAAATACAAGACAGCCAATTAGAAATGTC...TAA')

CDS
join{[67368:67482](-), [95562:95793](-), [95000:95027](-)}
ID: <unknown

In [16]:
import pandas as pd
import numpy as np
from Bio import SeqRecord, SeqIO
from Bio.SeqFeature import SeqFeature, SimpleLocation
import os

# variables
c_cal = "plastomes/final/Crepis_callicephala.gb"
c_pur = "plastomes/final/Crepis_purpurea.gb"
cich = "plastomes/test/cichorieae_seqrecords.gb"


# Do initial parsing
for acc in [cich]:
    print(acc)
    
    with open(acc, "r") as handle:
        for record in SeqIO.parse(handle, "genbank"):
            if "sativa" in record.annotations.get('organism', "") and "NC_" in record.id:
                features = record.features
                print("\n", record.id, record.annotations.get('organism', ""), len(features) - 1)
                
                # check if there are features other than 'source'
                if len(features) > 1:
                    for feature in features:
                        if feature.qualifiers.get('gene', [""])[0] == "rps12":
                            #print(f'{feature.location}\n')
                            #print(feature.translate(record, table=11))
                            ex = feature.extract(record)
                            print(f'\n{feature.type}\n{feature.location}\n{ex}')

plastomes/test/cichorieae_seqrecords.gb

 NC_061693.1 Lactuca sativa var. angustana 262

gene
join{[69571:69685](-), [97777:98020](-)}
ID: <unknown id>
Name: <unknown name>
Description: <unknown description>
Number of features: 0
Seq('ATGCCAACGATTAAACAACTTATTAGAAATACAAGACAGCCAATTAGAAATGTC...TAG')

CDS
join{[69571:69685](-), [97777:98020](-)}
ID: <unknown id>
Name: <unknown name>
Description: <unknown description>
Number of features: 0
Seq('ATGCCAACGATTAAACAACTTATTAGAAATACAAGACAGCCAATTAGAAATGTC...TAG')

gene
join{[69571:69685](+), [138808:139051](+)}
ID: NC_061693.1
Name: NC_061693
Description: Lactuca sativa var. angustana chloroplast, complete genome
Number of features: 0
/molecule_type=DNA
Seq('ATACACCCTAGTACATGTTCCTCGACGCTGAGGGCATCCCCGAAGCGCGGGGGA...TAG')

CDS
join{[69571:69685](+), [138808:139051](+)}
ID: NC_061693.1
Name: NC_061693
Description: Lactuca sativa var. angustana chloroplast, complete genome
Number of features: 0
/molecule_type=DNA
Seq('ATACACCCTAGTACATGTTCCTCGACGCTGAGG

In [19]:
import pandas as pd
import numpy as np
from Bio import SeqRecord, SeqIO
from Bio.SeqFeature import SeqFeature, SimpleLocation
import os

# variables
c_cal = "plastomes/final/Crepis_callicephala.gb"
c_pur = "plastomes/final/Crepis_purpurea.gb"
cich = "plastomes/test/cichorieae_seqrecords.gb"
a_thal = "plastomes/refs/arabidopsis_thaliana.gb"
n_tab = "plastomes/refs/nicotiana_tabacum.gb"


# Do initial parsing
for acc in [a_thal, n_tab]:
    print(acc)
    
    with open(acc, "r") as handle:
        for record in SeqIO.parse(handle, "genbank"):
            if "NC_" in record.id:
                features = record.features
                print("\n", record.id, record.annotations.get('organism', ""), len(features) - 1)
                
                # check if there are features other than 'source'
                if len(features) > 1:
                    for feature in features:
                        if feature.qualifiers.get('gene', [""])[0] == "rps12":
                            #print(f'{feature.location}\n')
                            #print(feature.translate(record, table=11))
                            ex = feature.extract(record)
                            print(f'\n{feature.type}\n{feature.location}\n{ex}')

plastomes/refs/arabidopsis_thaliana.gb

 NC_001879.2 Nicotiana tabacum 368

gene
join{[72223:72337](-), [100061:100855](-)}
ID: <unknown id>
Name: <unknown name>
Description: <unknown description>
Number of features: 4
Seq('ATGCCAACTATTAAACAACTTATTAGAAATACAAGACAGCCAATCAGAAATGTC...TAA')

CDS
join{[72223:72337](-), [100623:100855](-), [100061:100087](-)}
ID: <unknown id>
Name: <unknown name>
Description: <unknown description>
Number of features: 3
Seq('ATGCCAACTATTAAACAACTTATTAGAAATACAAGACAGCCAATCAGAAATGTC...TAA')

gene
join{[72223:72337](-), [141774:142568](+)}
ID: <unknown id>
Name: <unknown name>
Description: <unknown description>
Number of features: 3
Seq('ATGCCAACTATTAAACAACTTATTAGAAATACAAGACAGCCAATCAGAAATGTC...TAA')

CDS
join{[72223:72337](-), [141774:142006](+), [142542:142568](+)}
ID: <unknown id>
Name: <unknown name>
Description: <unknown description>
Number of features: 3
Seq('ATGCCAACTATTAAACAACTTATTAGAAATACAAGACAGCCAATCAGAAATGTC...TAA')

exon
[72223:72337](-)
ID: <unknown id>